In [ ]:
import pandas as pd


In [ ]:
from google.colab import files
uploaded1 = files.upload()

Saving data.csv to data.csv


In [ ]:
import io
data = pd.read_csv(io.BytesIO(uploaded1['data.csv']))

In [ ]:
data

,date,text,label,tokens_without_sw_stem
0,2021-08-17 06:28:35,we get your concern regarding the delivery of...,Neutral,"['get', 'concern', 'regard', 'deliveri', 'orde..."
1,2021-08-16 22:31:19,mercykxoxo alexa play im so excited Visage en ...,Positive,"['mercykxoxo', 'alexa', 'play', 'excit', 'visa..."
2,2021-08-16 22:31:16,itsashleeg self care is the best care Cœur noi...,Positive,"['itsashleeg', 'self', 'best', 'cœur', 'noir',..."
3,2021-08-16 19:05:00,jenpilarski thank you for this amazing opportu...,Positive,"['jenpilarski', 'thank', 'amaz', 'opportun', '..."
4,2021-08-16 19:00:00,its nationaltellajokeday – which you definitel...,Neutral,"['nationaltellajokeday', '–', 'definit', 'cant..."
...,...,...,...,...
3195,2021-02-12 17:44:02,sportytechmama wow your crafting skills are re...,Positive,"['sportytechmama', 'wow', 'craft', 'skill', 'r..."
3196,2021-02-12 17:41:01,callmephe awesome we got your list books to ma...,Positive,"['callmeph', 'awesom', 'got', 'list', 'book', ..."
3197,2021-02-12 17:27:02,thanks for sending in your infurmation so fas...,Positive,"['thank', 'send', 'infurm', 'fast', 'incom', '..."
3198,2021-02-12 17:22:53,were so excited you enjoyed the surprises Ope...,Positive,"['excit', 'enjoy', 'surpris', 'open', 'book', ..."


In [ ]:
from sklearn.model_selection import train_test_split
# Train Test Split Function
def split_train_test(top_data_df_small, test_size=0.3, shuffle_state=True):
    X_train, X_test, Y_train, Y_test = train_test_split(data[['date', 'text','tokens_without_sw_stem']], 
                                                        data['label'], 
                                                        shuffle=shuffle_state,
                                                        test_size=test_size, 
                                                        random_state=15)
    print("Value counts for Train sentiments")
    print(Y_train.value_counts())
    print("Value counts for Test sentiments")
    print(Y_test.value_counts())
    print(type(X_train))
    print(type(Y_train))
    X_train = X_train.reset_index()
    X_test = X_test.reset_index()
    Y_train = Y_train.to_frame()
    Y_train = Y_train.reset_index()
    Y_test = Y_test.to_frame()
    Y_test = Y_test.reset_index()
    print(X_train.head())
    return X_train, X_test, Y_train, Y_test

In [ ]:
# Call the train_test_split
X_train, X_test, Y_train, Y_test = split_train_test(data)

Value counts for Train sentiments
Positive    1694
Neutral      372
Negative     174
Name: label, dtype: int64
Value counts for Test sentiments
Positive    716
Neutral     166
Negative     78
Name: label, dtype: int64
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>
   index  ...                             tokens_without_sw_stem
0   1683  ...  ['your', 'welcom', '🖍️', 'delight', 'could', '...
1   1381  ...  ['jeremysickel', 'talent', 'terrif', 'tremend'...
2    179  ...  ['realli', 'amaz', 'quickli', 'sleep', 'bag', ...
3   1268  ...  ['avilslar', 'noth', 'hit', 'like', 'ramen', '...
4    996  ...  ['well', 'suppos', 'your', 'cover', 'visag', '...

[5 rows x 4 columns]


In [ ]:
from gensim.models import Word2Vec
import time
# Skip-gram model (sg = 1)
size = 1000
window = 3
min_count = 1
workers = 3
sg = 1

word2vec_model_file = 'word2vec_' + str(size) + '.model'
start_time = time.time()
stemmed_tokens = pd.Series(data['tokens_without_sw_stem']).values
# Train the Word2Vec Model
w2v_model = Word2Vec(stemmed_tokens, min_count = min_count, size = size, workers = workers, window = window, sg = sg)
print("Time taken to train word2vec model: " + str(time.time() - start_time))
w2v_model.save(word2vec_model_file)

Time taken to train word2vec model: 4.434787750244141


In [ ]:
import numpy as np
# Load the model from the model file
sg_w2v_model = Word2Vec.load(word2vec_model_file)
# Get the mean for the vectors for an example review
print("Print the length after taking average of all word vectors in a sentence:")
print(np.mean([sg_w2v_model[token] for token in data['tokens_without_sw_stem'][0]], axis=0))

Print the length after taking average of all word vectors in a sentence:
[-1.17392447e-02 -1.73539501e-02 -5.17472513e-02 -1.50834722e-02
  1.55202067e-02  4.64175008e-02  3.63387764e-02 -3.42517979e-02
 -4.10515629e-02 -1.31226163e-02 -1.65311489e-02  8.12627189e-03
  9.20905545e-02  2.45113242e-02  4.21098433e-02  4.89722900e-02
 -3.01639754e-02 -5.89678437e-03  6.75403625e-02  4.93804030e-02
  1.42397285e-02 -3.36543731e-02  6.02852926e-02  1.44143980e-02
  4.56909612e-02 -1.80565603e-02  5.42814508e-02 -3.26942392e-02
  5.34751127e-03 -3.89999002e-02  1.82375479e-05 -1.12784014e-03
 -2.42315307e-02 -2.00369861e-02  2.70974752e-03  4.35418412e-02
  1.90610271e-02  3.99614591e-03  1.96870621e-02 -3.43720019e-02
 -2.80105844e-02 -2.87823603e-02 -3.26581635e-02  5.60421729e-03
 -2.34752316e-02  4.41518351e-02 -2.08987780e-02  2.13458370e-02
  6.12474140e-03  2.50323750e-02 -1.20720714e-02 -6.09064214e-02
 -1.62968747e-02 -5.06320212e-04  1.81762408e-02 -1.23882629e-02
  1.09889302e-02 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [ ]:
# Store the vectors for train data in following file
word2vec_filename = 'train_review_word2vec.csv'
with open(word2vec_filename, 'w+') as word2vec_file:
    for index, row in X_train.iterrows():
        model_vector = (np.mean([sg_w2v_model[token] for token in row['tokens_without_sw_stem']], axis=0)).tolist()
        if index == 0:
            header = ",".join(str(ele) for ele in range(1000))
            word2vec_file.write(header)
            word2vec_file.write("\n")
        # Check if the line exists else it is vector of zeros
        if type(model_vector) is list:  
            line1 = ",".join( [str(vector_element) for vector_element in model_vector] )
        else:
            line1 = ",".join([str(0) for i in range(1000)])
        word2vec_file.write(line1)
        word2vec_file.write('\n')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


In [ ]:
import time
#Import the DecisionTreeeClassifier
from sklearn.tree import DecisionTreeClassifier
# Load from the filename
word2vec_df = pd.read_csv(word2vec_filename)
#Initialize the model
clf_decision_word2vec = DecisionTreeClassifier()

start_time = time.time()
# Fit the model
clf_decision_word2vec.fit(word2vec_df, Y_train['label'])
print("Time taken to fit the model with word2vec vectors: " + str(time.time() - start_time))

Time taken to fit the model with word2vec vectors: 5.857373237609863


In [ ]:
from sklearn.metrics import classification_report
test_features_word2vec = []
for index, row in X_test.iterrows():
    model_vector = np.mean([sg_w2v_model[token] for token in row['tokens_without_sw_stem']], axis=0)
    if type(model_vector) is list:
        test_features_word2vec.append(model_vector)
    else:
        test_features_word2vec.append(np.array([0 for i in range(1000)]))
test_predictions_word2vec = clf_decision_word2vec.predict(test_features_word2vec)
print(classification_report(Y_test['label'],test_predictions_word2vec))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.


              precision    recall  f1-score   support

    Negative       0.00      0.00      0.00        78
     Neutral       0.00      0.00      0.00       166
    Positive       0.75      1.00      0.85       716

    accuracy                           0.75       960
   macro avg       0.25      0.33      0.28       960
weighted avg       0.56      0.75      0.64       960



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
